### Clustering Quality Measures
#### Clustering quality
    There are majorly two types of measures to assess the clustering performance. 
    (i) Extrinsic Measures which require ground truth labels. 
    Examples are Adjusted Rand index, Fowlkes-Mallows scores, Mutual information based scores, 
    Homogeneity, Completeness and V-measure
 

### NMI (Normal Mutual Information)

    This metric is furthermore symmetric: 
    1. switching label_true with label_pred will return the same score value. 
       This can be useful to measure the agreement of two independent label assignments strategies 
       on the same dataset when the real ground truth is not known.
    
    sklearn.metrics.normalized_mutual_info_score(labels_true, labels_pred, *, average_method='arithmetic')
    
    Parameters
        1. labels_trueint array, shape = [n_samples]
            A clustering of the data into disjoint subsets.
        
        2. labels_predint array-like of shape (n_samples,)
            A clustering of the data into disjoint subsets.
        
        3. average_methodstr, default=’arithmetic’
            How to compute the normalizer in the denominator. 
            Possible options are ‘min’, ‘geometric’, ‘arithmetic’, and ‘max’.
            Changed in version 0.22: The default value of average_method changed from 
            ‘geometric’ to ‘arithmetic’.

    Returns: nmifloat
        score between 0.0 and 1.0. 
        1.0 stands for perfectly complete labeling

In [2]:
from sklearn.metrics.cluster import normalized_mutual_info_score
normalized_mutual_info_score([0, 0, 1, 1], [0, 0, 1, 1])

1.0

In [3]:
normalized_mutual_info_score([0, 0, 1, 1], [1, 1, 0, 0])

1.0

### Adjusted Random Score 

   1. The adjusted Rand index is thus ensured to have a value close to 0.0 for random labeling 
   independently of the number of clusters and samples and exactly 1.0 when the clusterings 
   are identical (up to a permutation). 
   2. ARI is a symmetric measure:

   
    sklearn.metrics.adjusted_rand_score(labels_true, labels_pred)
    
    Paramters:
    1. labels_true int array, shape = [n_samples]
        Ground truth class labels to be used as a reference 
    
    2. labels_predarray-like of shape (n_samples,)
        Cluster labels to evaluate

    Returns: ARIfloat

    Similarity score between -1.0 and 1.0. 
    Random labelings have an ARI close to 0.0. 
    1.0 stands for perfect match.

In [4]:
from sklearn.metrics.cluster import adjusted_rand_score
adjusted_rand_score([0, 0, 1, 1], [0, 0, 1, 1])

1.0

In [5]:
adjusted_rand_score([0, 0, 1, 1], [1, 1, 0, 0])

1.0

    1. True Positives are on the diagonal position
    2. False positives are column-wise sums. Without the diagonal
    3. False negatives are row-wise sums. Without the diagonal.


In [6]:
import numpy as np

In [7]:
def accuracy(cm):
    """computes accuracy from confusion matrix"""
    return np.trace(cm, dtype=float) / np.sum(cm)


In [8]:
def precision(cm):
    """computes precision from confusion matrix"""
    true_pos = np.diag(cm) #np.trace(cm, dtype=float) is same as diag
    false_pos = np.sum(cm, axis=0) - true_pos
    false_neg = np.sum(cm, axis=1) - true_pos

    return np.sum(true_pos) / np.sum(true_pos + false_pos)


In [9]:
#print("---------------------\nnew confusion matrix:\n %s\naccuracy: %.2f" % (str(new_cm), precision(new_cm)))

In [10]:
def recall(cm):
    """computes recall from confusion matrix"""
    true_pos = np.diag(cm) #np.trace(cm, dtype=float) is same as diag
    false_pos = np.sum(cm, axis=0) - true_pos
    false_neg = np.sum(cm, axis=1) - true_pos

    return np.sum(true_pos) / np.sum(true_pos + false_neg)


In [11]:
#print("---------------------\nnew confusion matrix:\n %s\naccuracy: %.2f" % (str(new_cm), recall(new_cm)))

In [12]:
def fMeasure(cm, f=1):
    """computes F-Measure from confusion matrix"""
    true_pos = np.diag(cm) #np.trace(cm, dtype=float) is same as diag
    false_pos = np.sum(cm, axis=0) - true_pos
    false_neg = np.sum(cm, axis=1) - true_pos
    
    p = np.sum(true_pos) / np.sum(true_pos + false_pos)
    r = np.sum(true_pos) / np.sum(true_pos + false_neg)
    
    f_square = f**2
    return (1 + f_square)*(p*r)/(f_square * (p + r))


In [13]:
#print("---------------------\nnew confusion matrix:\n %s\naccuracy: %.2f" % (str(new_cm), fMeasure(new_cm)))

In [14]:
'''
If we use (external) classification evalutation measures like F1 or 
accuracy for clustering evaluation, problems may arise. 

One way to fix is to perform label matching.

Here we performs kmeans clustering on the Iris dataset and proceed to use 
the Hungarian (Munkres) algorithm to correct the mismatched labeling. 
'''
#!pip install munkres

import sys
import numpy as np
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.metrics import confusion_matrix

from munkres import Munkres

def make_cost_matrix(c1, c2):
    """
    """
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    assert(l1 == l2 and np.all(uc1 == uc2))

    m = np.ones([l1, l2])
    for i in range(l1):
        it_i = np.nonzero(c1 == uc1[i])[0]
        for j in range(l2):
            it_j = np.nonzero(c2 == uc2[j])[0]
            m_ij = np.intersect1d(it_j, it_i)
            m[i,j] =  -m_ij.size
    return m

def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])


In [18]:
"""entry point"""
dataset = datasets.load_iris() # loads the iris dataset
data, classes = dataset.data, dataset.target # data and true labels
algo = KMeans(n_clusters=3, random_state = 0)

labels = algo.fit(data).labels_ # performs the algo and get the predicted labels
num_labels = len(np.unique(classes))

cm = confusion_matrix(classes, labels, labels=range(num_labels)) # gets the confusion matrix
print("---------------------\nold confusion matrix:\n %s\naccuracy: %.2f" % (str(cm), accuracy(cm)))

print("labels: ", len(labels), type(labels))
print("classes: ", len(classes), type(classes))
cost_matrix = make_cost_matrix(labels, classes)

m = Munkres()
indexes = m.compute(cost_matrix)
mapper = { old: new for (old, new) in indexes }

print("---------------------\nmapping:")
for old, new in mapper.items():
    print("map: %s --> %s" %(old, new))

new_labels = translate_clustering(labels, mapper)
new_cm = confusion_matrix(classes, new_labels, labels=range(num_labels))


---------------------
old confusion matrix:
 [[ 0 50  0]
 [ 2  0 48]
 [36  0 14]]
accuracy: 0.09
labels:  150 <class 'numpy.ndarray'>
classes:  150 <class 'numpy.ndarray'>
---------------------
mapping:
map: 0 --> 2
map: 1 --> 0
map: 2 --> 1


In [16]:
print("---------------------\nnew confusion matrix:\n %s\naccuracy: %.2f" % (str(new_cm), accuracy(new_cm)))

---------------------
new confusion matrix:
 [[50  0  0]
 [ 0 48  2]
 [ 0 14 36]]
accuracy: 0.89


In [81]:
print("---------------------\nnew confusion matrix:\n %s\naccuracy: %.2f" % (str(new_cm), precision(new_cm)))

---------------------
new confusion matrix:
 [[50  0  0]
 [ 0 48  2]
 [ 0 14 36]]
accuracy: 0.89


In [82]:
print("---------------------\nnew confusion matrix:\n %s\naccuracy: %.2f" % (str(new_cm), recall(new_cm)))

---------------------
new confusion matrix:
 [[50  0  0]
 [ 0 48  2]
 [ 0 14 36]]
accuracy: 0.89


In [83]:
print("---------------------\nnew confusion matrix:\n %s\naccuracy: %.2f" % (str(new_cm), fMeasure(new_cm)))

---------------------
new confusion matrix:
 [[50  0  0]
 [ 0 48  2]
 [ 0 14 36]]
accuracy: 0.89
